In [0]:
import unicodedata
from pyspark.sql import functions
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.functions import col, count, months_between, current_date, date_format, to_timestamp, lpad, regexp_replace

In [0]:
df_cliente = spark.createDataFrame(
    [
        ("001", "José", "Anápolis", "São Paulo", "01-09-1900"),
        ("02", "Igor", "Anápolis", "São Paulo", "11-09-1977"),
        ("3", "Leonardo", "Anápolis", "São Paulo", "21-12-2000"),
        ("04", "Humberto", "Pato Branco", "Rio Grande do Sul", "13-11-1964"),
        ("005", "Isaias", "Pato Branco", "Rio Grande do Sul", "07-07-2002"),
        ("6", "Lucas", "Taua", "Ceará", "05-09-1984")
    ],
    ["cod_cliente", "Nome", "Município", "Estado", "data_nascimento"]
)

In [0]:
colunas = ["cod_cliente", "Nome", "Município", "Estado", "data_nascimento"]
linha = spark.createDataFrame([(-1,-1,-1,-1,-1)], colunas)
appended = df_cliente.union(linha)
appended.show()

+-----------+--------+-----------+-----------------+---------------+
|cod_cliente|    Nome|  Município|           Estado|data_nascimento|
+-----------+--------+-----------+-----------------+---------------+
|        001|    José|   Anápolis|        São Paulo|     01-09-1900|
|         02|    Igor|   Anápolis|        São Paulo|     11-09-1977|
|          3|Leonardo|   Anápolis|        São Paulo|     21-12-2000|
|         04|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|
|        005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|
|          6|   Lucas|       Taua|            Ceará|     05-09-1984|
|         -1|      -1|         -1|               -1|             -1|
+-----------+--------+-----------+-----------------+---------------+



In [0]:
WindowSpec = Window.partitionBy("Município")
df_cliente = df_cliente.withColumn("qtd_municipio" ,
                                count("Município").over(WindowSpec))
df_cliente = df_cliente.sort("Estado")

In [0]:
df_cliente = df_cliente.withColumn('Idade', (months_between(current_date(), to_timestamp(col("data_nascimento"),'dd-MM-yyyy')) / 12).cast('int'))

In [0]:
df_cliente = df_cliente.withColumn('cod_cliente', lpad(col("cod_cliente"), 3, '0'))

In [0]:
df_cliente = df_cliente.withColumn('data_atualizacao', current_date())

In [0]:
# A função remove os caracteres especiais
def ascii_ignore(x):
    return x.encode('ascii', 'ignore').decode('ascii')

ascii_udf = udf(ascii_ignore)

df_cliente.withColumn("Estado", ascii_udf('Estado')).show()

+-----------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|cod_cliente|    Nome|  Município|           Estado|data_nascimento|qtd_municipio|Idade|data_atualizacao|
+-----------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|        006|   Lucas|       Taua|             Cear|     05-09-1984|            1|   37|      2022-07-14|
|        004|Humberto|Pato Branco|Rio Grande do Sul|     13-11-1964|            2|   57|      2022-07-14|
|        005|  Isaias|Pato Branco|Rio Grande do Sul|     07-07-2002|            2|   20|      2022-07-14|
|        001|    José|   Anápolis|         So Paulo|     01-09-1900|            3|  121|      2022-07-14|
|        002|    Igor|   Anápolis|         So Paulo|     11-09-1977|            3|   44|      2022-07-14|
|        003|Leonardo|   Anápolis|         So Paulo|     21-12-2000|            3|   21|      2022-07-14|
+-----------+--------+-----------+------------

In [0]:
# Remove acentos
from pyspark.sql import functions as F

df_cliente\
    .withColumn('Estado', F.translate('Estado',
                                      'ãäöüẞáäčďéěíĺľňóôŕšťúůýžÄÖÜẞÁÄČĎÉĚÍĹĽŇÓÔŔŠŤÚŮÝŽ',
                                      'aaousaacdeeillnoorstuuyzAOUSAACDEEILLNOORSTUUYZ'))\
    .show(truncate=False)

+-----------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|cod_cliente|Nome    |Município  |Estado           |data_nascimento|qtd_municipio|Idade|data_atualizacao|
+-----------+--------+-----------+-----------------+---------------+-------------+-----+----------------+
|006        |Lucas   |Taua       |Ceara            |05-09-1984     |1            |37   |2022-07-14      |
|004        |Humberto|Pato Branco|Rio Grande do Sul|13-11-1964     |2            |57   |2022-07-14      |
|005        |Isaias  |Pato Branco|Rio Grande do Sul|07-07-2002     |2            |20   |2022-07-14      |
|001        |José    |Anápolis   |Sao Paulo        |01-09-1900     |3            |121  |2022-07-14      |
|002        |Igor    |Anápolis   |Sao Paulo        |11-09-1977     |3            |44   |2022-07-14      |
|003        |Leonardo|Anápolis   |Sao Paulo        |21-12-2000     |3            |21   |2022-07-14      |
+-----------+--------+-----------+------------

In [0]:
def removes (string: str) -> str:
    normalized = unicodedata.normalize('NFD',string)
    return normalized.encode('ascii','ignore').decode('utf8').casefold()

removeudf = udf(lambda z: removes(z))

#ja removendo a coluna
df_cliente = df_cliente.withColumn("Estado",removeudf(col('Estado')))

In [0]:
display(df_cliente)

cod_cliente,Nome,Município,Estado,data_nascimento,qtd_municipio,Idade,data_atualizacao
006,Lucas,Taua,ceara,05-09-1984,1,37,2022-07-14
004,Humberto,Pato Branco,rio grande do sul,13-11-1964,2,57,2022-07-14
005,Isaias,Pato Branco,rio grande do sul,07-07-2002,2,20,2022-07-14
001,José,Anápolis,sao paulo,01-09-1900,3,121,2022-07-14
002,Igor,Anápolis,sao paulo,11-09-1977,3,44,2022-07-14
003,Leonardo,Anápolis,sao paulo,21-12-2000,3,21,2022-07-14


In [0]:
df_cliente = df_cliente.withColumn('Estado',regexp_replace(col("Estado"),lista_caracteres[i],lista_caracteres[i + 1]))

In [0]:
display(df_cliente)

cod_cliente,Nome,Município,Estado,data_nascimento,qtd_municipio,Idade,data_atualizacao
006,Lucas,Taua,Ceara,05-09-1984,1,37,2022-01-21
004,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,2,57,2022-01-21
005,Isaias,Pato Branco,Rio Grande do Sul,07-07-2002,2,19,2022-01-21
001,José,Anápolis,Sao Paulo,01-09-1900,3,121,2022-01-21
002,Igor,Anápolis,Sao Paulo,11-09-1977,3,44,2022-01-21
003,Leonardo,Anápolis,Sao Paulo,21-12-2000,3,21,2022-01-21
